In [2]:
import praw # Python Reddit API Wrapper
#Praw documentation: https://praw.readthedocs.io/en/stable/index.html
import pandas as pd


# Create a Reddit instance
# This is a read-only instance since we only need to read data
# We don't need it to be authenticated since we aren't posting anything
reddit = praw.Reddit(
    client_id='51f3IBDpHGRzaGh0FGj0ng',
    client_secret='C-mvx1K5Flj_4kM-Grc9FLT2_bHxag',
    user_agent='TradingAnalysisApp by /u/Ok_Ability_9624',
)



Version 7.8.0 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


In [3]:
print(reddit.read_only)

True


In [4]:
# Fetch and print the titles of the top 10 hot posts in the 'test' subreddit
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)

Some test commands
Test
Bhad Bhabie - Sexy Pink Nipple

This is a test post
test post
test
This is a test post
My Video Title
New post test comment 12/11/2024 09:41:43
New post test comment 12/11/2024 08:00:22


In [5]:
subreddit = reddit.subreddit("wallstreetbets")
print(subreddit.display_name)  
print(subreddit.title)
print(subreddit.description)

wallstreetbets
wallstreetbets
The rules and submission guidelines are maintained on new Reddit so be sure to check them and make sure you're up to date.

* Read the [rules](https://www.reddit.com/r/wallstreetbets/about/rules)

* Read the [comment and submission guide](https://www.reddit.com/r/wallstreetbets/wiki/contentguide)

* Read the [FAQ](https://www.reddit.com/r/wallstreetbets/wiki/faq) if you're new to both wallstreetbets and trading.

---
**Join the discord**

[WSB Discord](https://discord.gg/wsbverse)

**filter by flairs**

^Navigate ^WSB |^We ^recommend ^best ^daily ^DD
:--|:--     
**DD** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restrict_sr=on&q=flair%3ADD) / [**Best Daily**](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=day) / [Best Weekly](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=week)
**Discussion** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restric

In [ ]:

stockCSV = pd.read_csv('stock_info.csv') #Read the CSV file

stockCSV['Ticker'] = '$' + stockCSV['Ticker'] #Add a $ sign in front of the ticker
stockCSV.head() #Display the first 5 rows of the dataframe


,Ticker,Name,Exchange
0,$A,Agilent Technologies,NYSE
1,$AA,Alcoa Inc.,NYSE
2,$AAN,Aaron's Inc,NYSE
3,$AAT,American Assets Trust,NYSE
4,$AAV,Advantage Oil & Gas Ltd,NYSE


In [16]:

postData = []
import nltk  #Natural Language Toolkit
nltk.download('punkt') #sentence word tokenziation
import textstat #a simple python library to calculate statistics from text 
from textblob import TextBlob
from nltk.tokenize import TabTokenizer
import datetime
buySell = 0

tokenizer = TabTokenizer() #tab tokenizer

# Fetch the top 1000 hot posts in the 'wallstreetbets' subreddit
for submission in subreddit.hot(limit=1000):
        #textWords = nltk.word_tokenize(submission.selftext) #tokenize the post text, pretty sure we do not need this
        wordsPost = submission.selftext.split() #split the title into words
        blob = TextBlob(submission.selftext, tokenizer=tokenizer) #create a textblob object
        print(blob.tokens) #print the sentiment polarity
        
        #determining buy sell depending on sentiment
        if(blob.sentiment.polarity >0):
            buySell = 1
        elif(blob.sentiment.polarity <0):
            buySell = -1
        else:
            buySell = 0
        
        if(submission.selftext == ''): #skip any posts that contain no text
            continue
        elif any(word in stockCSV['Ticker'].values for word in wordsPost): #check if any of the words in the post are stock abbreviations
            postData.append({
                 #These are indentifying data
                'stock abbreviation': next(word for word in wordsPost if word in stockCSV['Ticker'].values),
                'stock name': stockCSV.loc[stockCSV['Ticker'] == next(word for word in wordsPost if word in stockCSV['Ticker'].values)]['Name'].values[0],
                'title': submission.title,
                'date': datetime.datetime.fromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'id': submission.id,
                'url': submission.url, 
                'post text': submission.selftext, #post text
                #All of the following are features that can be used to predict the stock price
                'upvote Ratio': submission.upvote_ratio, #upvote ratio
                'comment number': submission.num_comments, #number of comments
                'text length': len(submission.selftext),#length of the post text
                'flesh_kincaid_grade': textstat.flesch_kincaid_grade(submission.selftext), #Flesch Kincaid Grade
                'lexical_diversity': len(set(wordsPost)) / len(wordsPost), #lexical diversity
                'Sentiment (polarity):': blob.sentiment.polarity, #Sentiment
                'Sentiment (subjectivity)': blob.sentiment.subjectivity, #Subjectivity
                'Buy/Sell': buySell #Buy or Sell
            })
        else:
            continue
        

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['This post contains content not supported on old Reddit. [Click here to view the full post](https://sh.reddit.com/r/wallstreetbets/comments/1gqncsa)']
['']
['I made this play… And it has not worked out… Any advice? \n\nHere’s my theory though: Spirit Airlines is aware that Trump is mindful of their financial situation. Joe Biden’s DOJ sued and blocked JetBlue’s acquisition of Spirit Airlines in March 2023. The sole purpose of blocking this merger was to preserve competition in the low-cost airline market. \n\n“As our complaint alleges, the merger of JetBlue and Spirit would result in higher fares and fewer choices for tens of millions of travelers, with the greatest impact felt by those who rely on what are known as ultra-low-cost carriers in order to fly,” said Attorney General Merrick B. Garland. “Companies in every industry should understand by now that this Justice Department will not hesitate to enforce our antitrust laws and protect American consumers.”\n\nMaybe, just maybe, thi

In [17]:
df = pd.DataFrame(postData)
df 

,stock abbreviation,stock name,title,date,id,url,post text,upvote Ratio,comment number,text length,flesh_kincaid_grade,lexical_diversity,Sentiment (polarity):,Sentiment (subjectivity),Buy/Sell
0,$TSLA,Tesla Motors,I ded it agan,2024-11-13 10:57:23,1gqg3wx,https://www.reddit.com/gallery/1gqg3wx,Hello brother degens. Well essentially I got h...,0.95,226,1068,5.0,0.748691,0.037255,0.374510,1
1,$F,Ford Motor Company,Fell into the Ford trap... AGAIN,2024-11-13 13:33:41,1gqjwvb,https://www.reddit.com/r/wallstreetbets/commen...,I'm probably not the only one to do this multi...,0.82,47,435,9.0,0.771084,-0.011905,0.469048,-1
2,$RKT,Rock-Tenn Company,We done saved the farm boys,2024-11-12 16:42:38,1gpwda8,https://www.reddit.com/r/wallstreetbets/commen...,"Good job regards, now Gomer can save his 10 ac...",0.93,139,647,7.4,0.852632,0.135714,0.528571,1
3,$TSLA,Tesla Motors,Thank you Elon! $TSLA $10k to $149k in under a...,2024-11-11 18:28:35,1gp6l38,https://i.redd.it/k519nqm4xc0e1.jpeg,Opened a lotto account last week after the ele...,0.85,471,129,4.0,0.956522,0.175000,0.308333,1
4,$MSTR,Microstrategy Inc.,6 years of trading to break even…,2024-11-11 10:02:25,1gou6tr,https://i.redd.it/1qpznfqtea0e1.jpeg,I literally lost money for 5 years and decided...,0.97,205,217,7.3,0.875000,-0.025000,0.241667,-1
5,$INTC,Intel Corp.,Intel CEO optimistic about CHIPS Act’s future ...,2024-11-09 06:40:01,1gn8a0p,https://www.reddit.com/r/wallstreetbets/commen...,[https://www.oregonlive.com/silicon-forest/202...,0.94,121,541,33.1,0.878049,0.000000,0.100000,0
6,$INTC,Intel Corp.,Biden Admin Rushing to Distribute CHIPS Act Fu...,2024-11-08 17:59:36,1gmvo4c,https://www.bloomberg.com/news/articles/2024-1...,$INTC,0.96,564,5,-3.5,1.000000,0.000000,0.000000,0


In [19]:
#TO DO 
#1. Add Data from Yahoo Finance API to the dataframe
#2. Get the "buy/sell" value of that stock for the day
#3. See what happended a day later
#4. Get the Delta between the two values
#5. Use that to create a model that predicts the stock price
newBlob = TextBlob("Je ne sais pas comment je vais reussir mais je devrai pour mon pays ma mere ma pere pour mon dieu pour mon copine - Napolean 1805", tokenizer=tokenizer)
newBlob.tokens

WordList(['Je ne sais pas comment je vais reussir mais je devrai pour mon pays ma mere ma pere pour mon dieu pour mon copine - Napolean 1805'])